# Cracking Open the Hugging Face Transformers Library

Code authored by: Shawhin Talebi <br>
Blog: https://medium.com/towards-data-science/cracking-open-the-hugging-face-transformers-library-350aa0ef0161

### import modules

In [6]:
# %pip install torch
# %pip install gradio
import sys
sys.path.append( '/Users/hli36/anaconda3/pkgs' )

from transformers import pipeline, Conversation
import gradio as gr

ModuleNotFoundError: No module named 'gradio'

### code

#### Sentiment Analysis

In [4]:
# toy example 1
pipeline(task="sentiment-analysis")("Love this!")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'label': 'POSITIVE', 'score': 0.9998745918273926}]

In [5]:
# toy example 2
pipeline(task="sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")("Hate this!!!!!!")

[{'label': 'NEGATIVE', 'score': 0.9994969367980957}]

#### More Sentiment Analysis

In [9]:
# defining classifier
classifier = pipeline(task="sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")

In [10]:
classifier("Hate this.")

[{'label': 'NEGATIVE', 'score': 0.9997110962867737}]

In [11]:
# we can also pass in a list to classifier
text_list = ["This is great", \
             "Thanks for nothing", \
             "You've got to work on your face", \
             "You're beautiful, never change!"]

classifier(text_list)

[{'label': 'POSITIVE', 'score': 0.9998785257339478},
 {'label': 'POSITIVE', 'score': 0.9680058360099792},
 {'label': 'NEGATIVE', 'score': 0.87760990858078},
 {'label': 'POSITIVE', 'score': 0.9998120665550232}]

In [15]:
# if there are multiple target labels, we can return them all
from transformers import pipeline

classifier = pipeline(task="text-classification", model="SamLowe/roberta-base-go_emotions", top_k=None)


In [16]:
classifier(text_list[0])

[[{'label': 'admiration', 'score': 0.9526104927062988},
  {'label': 'approval', 'score': 0.030472081154584885},
  {'label': 'neutral', 'score': 0.015236238949000835},
  {'label': 'excitement', 'score': 0.006063766311854124},
  {'label': 'gratitude', 'score': 0.005296189337968826},
  {'label': 'joy', 'score': 0.004475208930671215},
  {'label': 'curiosity', 'score': 0.004322327673435211},
  {'label': 'realization', 'score': 0.004089603666216135},
  {'label': 'optimism', 'score': 0.0040772221982479095},
  {'label': 'disapproval', 'score': 0.004076563753187656},
  {'label': 'annoyance', 'score': 0.0035287411883473396},
  {'label': 'surprise', 'score': 0.0029730673413723707},
  {'label': 'disappointment', 'score': 0.002734639449045062},
  {'label': 'love', 'score': 0.0026945779100060463},
  {'label': 'amusement', 'score': 0.0024867437314242125},
  {'label': 'confusion', 'score': 0.002360738581046462},
  {'label': 'pride', 'score': 0.0021013382356613874},
  {'label': 'sadness', 'score': 0.00

#### Summarization

In [5]:
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

In [6]:
text = """
Hugging Face is an AI company that has become a major hub for open-source machine learning. 
Their platform has 3 major elements which allow users to access and share machine learning resources. 
First, is their rapidly growing repository of pre-trained open-source machine learning models for things such as natural language processing (NLP), computer vision, and more. 
Second, is their library of datasets for training machine learning models for almost any task. 
Third, and finally, is Spaces which is a collection of open-source ML apps.

The power of these resources is that they are community generated, which leverages all the benefits of open source i.e. cost-free, wide diversity of tools, high quality resources, and rapid pace of innovation. 
While these make building powerful ML projects more accessible than before, there is another key element of the Hugging Face ecosystem—their Transformers library.
"""
summarized_text = summarizer(text, min_length=5, max_length=140)[0]['summary_text']
summarized_text

'Hugging Face is an AI company that has become a major hub for open-source machine learning. They have 3 major elements which allow users to access and share machine learning resources.'

In [8]:
classifier(summarized_text)

NameError: name 'classifier' is not defined

#### Conversational

In [9]:
chatbot = pipeline(model="facebook/blenderbot-400M-distill")

RuntimeError: Instantiating a pipeline without a task set raised an error: (ProtocolError('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer')), '(Request ID: 085e7828-ef0f-48cd-8464-d1f26891f51f)')

In [4]:
conversation = chatbot("Hi I'm Shaw, how are you?")

NameError: name 'chatbot' is not defined

In [14]:
conversation

Conversation id: 5d4166e0-c207-4060-b1ae-253861d045ab 
user >> Hi I'm Shaw, how are you? 
bot >>  I'm doing well. How are you doing this evening? I just got home from work. 

In [15]:
conversation = chatbot("Where do you work?")

In [16]:
conversation

Conversation id: 5d4166e0-c207-4060-b1ae-253861d045ab 
user >> Hi I'm Shaw, how are you? 
bot >>  I'm doing well. How are you doing this evening? I just got home from work. 
user >> Where do you work? 
bot >>  I work at a grocery store. What about you? What do you do for a living? 

### Deploy Chatbot UI

#### Text Sentiment Chatbot

In [14]:


def top3_text_classes(message, history):
    return str(classifier(message)[0][:3]).replace('}, {', '\n').replace('[{', '').replace('}]', '')

demo_sentiment = gr.ChatInterface(top3_text_classes, title="Text Sentiment Chatbot", description="Enter your text, and the chatbot will classify the sentiment.")

demo_sentiment.launch()

UsageError: Line magic function `%import` not found.


#### Summarizer Chatbot

In [11]:
def summarizer_bot(message, history):
    return summarizer(message, min_length=5, max_length=140)[0]['summary_text']

demo_summarizer = gr.ChatInterface(summarizer_bot, title="Summarizer Chatbot", description="Enter your text, and the chatbot will return the summarized version.")

demo_summarizer.launch()

NameError: name 'gr' is not defined

#### Vanilla Chatbot

In [22]:
message_list = []
response_list = []

def vanilla_chatbot(message, history):
    conversation = chatbot(message)
    
    return conversation[0]['generated_text']

demo_chatbot = gr.ChatInterface(vanilla_chatbot, title="Vanilla Chatbot", description="Enter text to start chatting.")

demo_chatbot.launch()

Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.


test
[]
['tes']
[' Do you like teslas? They are a type of cylinder shaped object.']
test
[['tes', ' Do you like teslas? They are a type of cylinder shaped object.']]
['tes', 'Hello']
[' Do you like teslas? They are a type of cylinder shaped object.', ' Do you have a favorite type of tesla? They have been around since 1973.']
